(tutorial-model)=

# Build and fit models

## Approaches to Model Definition

Fundamentally, most pharmacokinetic models are ordinary differential equations (ODEs) characterizing the distribution of drug molecules. Therefore, when working with the Maspectra masmod framework, there are three methods available for constructing a Population Pharmacokinetic (PopPK) model:

1. **Use the built-in compartment model library:** Take advantage of the pre-existing library of compartment models.
2. **Closed-form equations:** Describe the structural model using closed-form solutions for the ODE equations.
3. **ODE models:** Define custom compartments and their associated ODEs, allowing for user-specific model configurations.

### Use the built-in compartment model library

The Maspectra model library provides implementations of the most commonly used PK compartment models, offering a combination of speed and user-friendliness.

The built-in compartment models are essentially pre-solved ODEs, where the compartments are predefined and cannot be modified.

Here is an example showcasing the usage of the compartment model library.

In [8]:
from mas.model import *

class DemoOralOneCmtModelWithBuiltInLibrary(pk.EvOneCmtLinear):
    def __init__(self) -> None:
        super().__init__()

        self.tv_cl = theta(0.2)
        self.tv_v = theta(9)
        self.tv_ka = theta(1.3)

        self.eta_cl = omega(0.1)
        self.eta_v = omega(0.1)
        self.eta_ka = omega(0.1)

        self.eps_prop = sigma(0.01)
        self.eps_add = sigma(0.15)

    def pred(self) -> Expression:
        cl = self.tv_cl * exp(self.eta_cl)
        v = self.tv_v * exp(self.eta_v)
        ka = self.tv_ka * exp(self.eta_ka)

        # to use the builtin PK library with the specified parameterization
        computed = self.trans2(cl=cl, v=v, ka=ka)

        ipred = computed.F
        y = ipred * (1 + self.eps_prop) + self.eps_add

        return y

### Closed-form equations

When we possess the closed-form equations for the ODEs, we have the option to directly incorporate them into our model. Here's an example illustrating this approach.

In [9]:
from mas.model import *

class DemoOralOneCmtModelWithClosedForm(Module):
    def __init__(self) -> None:
        super().__init__()

        self.tv_cl = theta(0.2)
        self.tv_v = theta(9)
        self.tv_ka = theta(1.3)

        self.eta_cl = omega(0.1)
        self.eta_v = omega(0.1)
        self.eta_ka = omega(0.1)

        self.eps_prop = sigma(0.01)
        self.eps_add = sigma(0.15)

        # retrieve the dose amounts from the ModelData 
        self.dose = column('dose')

    def pred(self) -> Expression:
        cl = self.tv_cl * exp(self.eta_cl)
        v = self.tv_v * exp(self.eta_v)
        ka = self.tv_ka * exp(self.eta_ka)

        k = cl / v

        # closed form equation for the solved ODEs
        conc = self.dose / v * ka / (ka - k) * (exp(-k * self.t) - exp(-ka * self.t))

        ipred = conc
        y = ipred * (1 + self.eps_prop) + self.eps_add

        return y

Please note that in this case, `dose` is provided as a data column. Therefore, it is essential to ensure that there is a corresponding column in each row to indicate the amount of the dose.

### ODE models

ODE models offer enhanced flexibility, particularly when dealing with complex or mechanism-related structural models.

In ODE models, the compartments are initially defined within the `__init__` method. It is possible to designate the default compartment for dosing and observation.

The ODEs themselves are defined within the `pred` method.

Below is an example showcasing an oral one-compartment model, same as the previously defined models.

In [10]:
from mas.model import *

class DemoOralOneCmtModelWithODE(Module):
    def __init__(self) -> None:
        super().__init__()

        self.tv_cl = theta(0.2)
        self.tv_v = theta(9)
        self.tv_ka = theta(1.3)

        self.eta_cl = omega(0.1)
        self.eta_v = omega(0.1)
        self.eta_ka = omega(0.1)

        self.eps_prop = sigma(0.01)
        self.eps_add = sigma(0.15)

        # definition of compartments
        self.cmt_depot = compartment(default_dose=True)
        self.cmt_central = compartment(default_obs=True)

    def pred(self) -> Expression:
        cl = self.tv_cl * exp(self.eta_cl)
        v = self.tv_v * exp(self.eta_v)
        ka = self.tv_ka * exp(self.eta_ka)

        k = cl / v

        # definition of ordinary differential equations 
        self.cmt_depot.dAdt = -ka * self.cmt_depot.A
        self.cmt_central.dAdt = ka * self.cmt_depot.A - k * self.cmt_central.A

        ipred = self.cmt_central.A / v
        y = ipred * (1 + self.eps_prop) + self.eps_add

        return y

## Model estimation

Maspectra supports various model estimation algorithms, including:

- Laplacian methods: FO/FOCEi/Laplace
- EM (Expectation-Maximization) methods: SAEM

Maspectra supports a range of minimization methods, including BOBYQA, Nelder-Mead, and BFGS, among others.

## Model diagnostics

To evaluate the model estimatio, you can utlize diagnostic plots:

- Goodness-of-fit plots: these can be generated using `FitResult.plot_goodness_of_fit()`
- Individual fit plots: these can be generated using `FitResult.plot_individual_fit()`

In addition, we offer commonly used diagnostic tools such as Visual Predictive Checks (VPC) and Bootstrap analysis.

See {ref}`case-warfarin` for example.